In [ ]:
import os
from scipy.io import mmread
from scipy.sparse import csr_matrix
import pandas as pd
import scanpy as sc
import numpy as np
import pickle
import resource

In [ ]:
cell = "multiome_for_uppmax"

In [ ]:
# paths
main_path = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600"
data_path = os.path.join(main_path, cell)
out_path = os.path.join(data_path, "scenicplus")
os.makedirs(out_path, exist_ok = True)
fragments_path = os.path.join(main_path, "fragments")
out_dir = "outs"
os.makedirs(out_dir, exist_ok = True)
mouse_general = os.path.join(main_path, "mouse_general")

In [ ]:
# Read metadata
metadata_file = os.path.join(data_path, 'metadata.tsv')
metadata = pd.read_csv(metadata_file, sep = '\t', index_col = 0)
print(metadata.head())
print(metadata.shape)

In [ ]:
# Load the RNA sparse matrix
atac_path = os.path.join(data_path, "atac_counts.mtx")
atac_matrix = mmread(atac_path).tocsr()

# Load the row and column names
genes_path = os.path.join(data_path, "regions.tsv")
cells_path = os.path.join(data_path, "cells.tsv")

genes = pd.read_csv(genes_path, header=None, sep='\t')[0].values
cells = pd.read_csv(cells_path, header=None, sep='\t')[0].values

atac_counts_df = pd.DataFrame.sparse.from_spmatrix(atac_matrix, index=genes, columns=cells)
print(atac_counts_df.head())

In [ ]:
# Load the RNA sparse matrix
rna_path = os.path.join(data_path, "rna_counts.mtx")
rna_matrix = mmread(rna_path).tocsr()

# Load the row and column names
genes_path = os.path.join(data_path, "gene_names.tsv")
cells_path = os.path.join(data_path, "cells.tsv")

genes = pd.read_csv(genes_path, header=None, sep='\t')[0].values
cells = pd.read_csv(cells_path, header=None, sep='\t')[0].values # even though it was read above :D

# Create a DataFrame if needed
rna_counts_df = pd.DataFrame.sparse.from_spmatrix(rna_matrix, index=genes, columns=cells)
# rna_counts_df.columns = rna_counts_df.columns.map(modify_row_names)
print(rna_counts_df.head())

In [ ]:
# creating adata object
adata = sc.AnnData(rna_counts_df.T, obs=metadata)
columns_to_keep = ['sample', 'experiment', 'condition', 'cell_ids', 'cluster_ids', 'merged_id']
adata.obs = adata.obs[columns_to_keep]

adata.obs['ds_sample'] = 'ds_sample'
# adata.obs.index = adata.obs.index + '_ds_sample'

adata.raw = adata.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
print(adata)

In [ ]:
# fragments_dict = {sample: os.path.join(fragments_path, sample, "atac_fragments.tsv.gz") for sample in adata.obs['sample'].unique()}
# print(fragments_dict)

In [ ]:
# # The following will run only once to generate subset-specific fragments file. If already generated, skip.

# columns = ['chrom', 'start', 'end', 'barcode', 'count']

# # An empty DataFrame to hold all fragments
# all_fragments_df = pd.DataFrame(columns=columns)

# # Group cells of interest by their sample IDs
# sample_to_cells = metadata.groupby('sample').apply(lambda x: x.index.tolist()).to_dict()
# print(sample_to_cells)

In [ ]:
# from concurrent.futures import ProcessPoolExecutor

# # Function to process each fragment file
# def process_sample(args):
#     sample, filepath, sample_to_cells, columns = args
#     print(f"Processing {sample} from {filepath}")
    
#     # Identify cells of interest for the current sample
#     cells_of_interest_sample = sample_to_cells.get(sample, [])
    
#     # Read the fragments file
#     fragments_df = pd.read_csv(filepath, sep='\t', compression='gzip', header=None, names=columns, comment='#')
    
#     # Prepend the sample ID to each cell barcode
#     fragments_df['barcode'] = sample + '_' + fragments_df['barcode']
    
#     # Filter the fragments to include only cells of interest for this sample
#     filtered_fragments_df = fragments_df[fragments_df['barcode'].isin(cells_of_interest_sample)]
    
#     return filtered_fragments_df

# # Prepare arguments for parallel processing
# args_list = [(sample, filepath, sample_to_cells, columns) for sample, filepath in fragments_dict.items()]

# # Parallel processing
# temp_results = []  # Store filtered DataFrames temporarily
# with ProcessPoolExecutor(max_workers=20) as executor:
#     results = executor.map(process_sample, args_list)
#     temp_results.extend(results)

# # Combine all filtered DataFrames
# all_fragments_df = pd.concat(temp_results, ignore_index=True)

# # Sort the combined DataFrame
# all_fragments_df = all_fragments_df.sort_values(by=['chrom', 'start'])

# # Write the subset DataFrame to a new uncompressed file
# all_fragments_df.to_csv(os.path.join(data_path, "subset_fragments.tsv"), sep='\t', header=False, index=False)

# print("All fragments combined and saved.")


In [ ]:
# print(all_fragments_df)

In [ ]:
# # then bash and run the following
# # bgzip -c /cfs/klemming/projects/supr/secilmis/naiss2023-23-600/multiome_for_uppmax/subset_fragments.tsv > /cfs/klemming/projects/supr/secilmis/naiss2023-23-600/multiome_for_uppmax/subset_fragments.tsv.gz
# # tabix -p bed /cfs/klemming/projects/supr/secilmis/naiss2023-23-600/multiome_for_uppmax/subset_fragments.tsv.gz

In [ ]:
# del(all_fragments_df)

In [ ]:
fragments_dict = {
    "ds_sample": os.path.join(data_path, "subset_fragments.tsv.gz")
}
print(fragments_dict)

In [ ]:
# Directory for output files
os.makedirs(os.path.join(out_dir, "consensus_peak_calling"), exist_ok=True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"), exist_ok=True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"), exist_ok=True)

# Load the mouse (mm10) chromosome sizes
chromsizes = pd.read_table(
    "http://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes",
    header=None,
    names=["Chromosome", "End"]
)
chromsizes.insert(1, "Start", 0)
# Check for spaces or non-printable characters in column names
chromsizes.columns = chromsizes.columns.str.strip()

# Check for spaces or non-printable characters in the data
chromsizes['Chromosome'] = chromsizes['Chromosome'].str.strip()

# Confirm data types
print(chromsizes.dtypes)

# Display first few rows to manually inspect
print(chromsizes.head())

In [ ]:
adata.obs['merged_id'] = adata.obs['merged_id'].astype(str)
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
bw_paths, bed_paths = export_pseudobulk(
    input_data=adata.obs,  
    variable='merged_id',  
    chromsizes=chromsizes,
    bed_path=os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"),
    bigwig_path=os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"),
    path_to_fragments=fragments_dict,
    sample_id_col='ds_sample',
    n_cpu=20,
    normalize_bigwig=True,
    split_pattern='--',  
    temp_dir=os.path.join(out_dir, "tmp")
)

In [ ]:
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv"), "wt") as f:
    for v in bw_paths:
        _ = f.write(f"{v}\t{bw_paths[v]}\n")
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv"), "wt") as f:
    for v in bed_paths:
        _ = f.write(f"{v}\t{bed_paths[v]}\n")

In [ ]:
bw_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bw_paths.update({v: p})
bed_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bed_paths.update({v: p})

In [ ]:
from pycisTopic.pseudobulk_peak_calling import peak_calling
macs_path = "macs2"

os.makedirs(os.path.join(out_dir, "consensus_peak_calling/MACS"), exist_ok = True)

narrow_peak_dict = peak_calling(
    macs_path = macs_path,
    bed_paths = bed_paths,
    outdir = os.path.join(out_dir, "consensus_peak_calling/MACS"),
    genome_size = 'mm',
    # n_cpu = 10,
    input_format = 'BEDPE',
    shift = 73,
    ext_size = 146,
    keep_dup = 'all',
    q_value = 0.05,
    _temp_dir = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/tmp"
)

In [ ]:
from pycisTopic.iterative_peak_calling import get_consensus_peaks
# Other param
peak_half_width=250
path_to_blacklist="/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/mouse_general/mm10-blacklist.v2.bed"
# Get consensus peaks
consensus_peaks = get_consensus_peaks(
    narrow_peaks_dict = narrow_peak_dict,
    peak_half_width = peak_half_width,
    chromsizes = chromsizes,
    path_to_blacklist = path_to_blacklist)

In [ ]:
consensus_peaks.to_bed(
    path = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed"),
    keep =True,
    compression = 'infer',
    chain = False)

In [ ]:
!pycistopic tss gene_annotation_list | grep Mouse

In [ ]:
!mkdir -p /cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc
!pycistopic tss get_tss \
    --output /cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc/tss.bed \
    --name "mmusculus_gene_ensembl" \
    --to-chrom-source ucsc \
    --ucsc mm10

In [ ]:
!head /cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc/tss.bed | column -t

In [ ]:
regions_path = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/consensus_peak_calling/consensus_regions.bed"
tss_path = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc/tss.bed"

# base output directory
output_dir = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc"

# loop through the fragments dictionary and run the pycistopic qc command
for sample, fragment_path in fragments_dict.items():
    output_path = os.path.join(output_dir, sample)
    os.makedirs(output_path, exist_ok=True)
    
    command = f"pycistopic qc --fragments {fragment_path} --regions {regions_path} --tss {tss_path} --output {output_path}"
    
    # run the command
    os.system(command)

In [ ]:
from pycisTopic.plotting.qc_plot import plot_sample_stats, plot_barcode_stats
import matplotlib.pyplot as plt

In [ ]:
for sample_id in fragments_dict:
    fig = plot_sample_stats(
        sample_id = sample_id,
        pycistopic_qc_output_dir = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc"
    )

In [ ]:
from pycisTopic.qc import get_barcodes_passing_qc_for_sample

In [ ]:
sample_id_to_barcodes_passing_filters = {}
sample_id_to_thresholds = {}
for sample_id in fragments_dict:
    (
        sample_id_to_barcodes_passing_filters[sample_id],
        sample_id_to_thresholds[sample_id]
    ) = get_barcodes_passing_qc_for_sample(
            sample_id = sample_id,
            pycistopic_qc_output_dir = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc",
            unique_fragments_threshold = 0, # use automatic thresholding
            tss_enrichment_threshold = 0, # use automatic thresholding
            frip_threshold = 0,
            use_automatic_thresholds = True,
    )

In [ ]:
for sample_id in fragments_dict:
    fig = plot_barcode_stats(
        sample_id = sample_id,
        pycistopic_qc_output_dir = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc",
        bc_passing_filters = sample_id_to_barcodes_passing_filters[sample_id],
        detailed_title = False,
        **sample_id_to_thresholds[sample_id]
    )

In [ ]:
# add pickle dump here
import pickle
pickle.dump(
    sample_id_to_barcodes_passing_filters,
    open(os.path.join(out_dir, "sample_id_to_barcodes_passing_filters.pkl"), "wb")
)

In [ ]:
import pickle
with open(os.path.join(out_dir, "sample_id_to_barcodes_passing_filters.pkl"), 'rb') as file:
    sample_id_to_barcodes_passing_filters = pickle.load(file)


In [ ]:
path_to_regions = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
path_to_blacklist = os.path.join(mouse_general, "mm10-blacklist.v2.bed")
pycistopic_qc_output_dir = "/cfs/klemming/projects/supr/secilmis/naiss2023-23-600/outs/qc"

from pycisTopic.cistopic_class import create_cistopic_object_from_fragments
import polars as pl

cistopic_obj_list = []
for sample_id in fragments_dict:
    sample_metrics = pl.read_parquet(
        os.path.join(pycistopic_qc_output_dir, f'{sample_id}.fragments_stats_per_cb.parquet')
    ).to_pandas().set_index("CB").loc[ sample_id_to_barcodes_passing_filters[sample_id] ]
    cistopic_obj = create_cistopic_object_from_fragments(
        path_to_fragments = fragments_dict[sample_id],
        path_to_regions = path_to_regions,
        path_to_blacklist = path_to_blacklist,
        metrics = sample_metrics,
        valid_bc = sample_id_to_barcodes_passing_filters[sample_id],
        # n_cpu = 1,
        project = sample_id,
        split_pattern = "--"
    )
    cistopic_obj_list.append(cistopic_obj)

In [ ]:
cistopic_obj = cistopic_obj_list[0]
print(cistopic_obj)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_path, "cistopic_obj.pkl"), "wb")
)

In [ ]:
# need to add adata index "--ds_sample"
print(adata.obs.head())
adata.obs.index = adata.obs.index + '--ds_sample'
print(adata.obs.head())
adata.write(os.path.join(out_path, "rna_adata.h5ad"))
# adata.write_h5ad(os.path.join(out_path, "rna_adata.h5ad"))

In [ ]:
cistopic_obj.add_cell_data(adata.obs)
print(cistopic_obj.cell_data.head())

In [ ]:
# save again
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_path, "cistopic_obj.pkl"), "wb")
)
# Load the pickled object
with open(os.path.join(out_path, "cistopic_obj.pkl"), "rb") as file:
    cistopic_obj = pickle.load(file)

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(cistopic_obj.fragment_matrix.T, expected_doublet_rate=0.1)
doublet_scores, predicted_doublets = scrub.scrub_doublets()
scrub.plot_histogram();
scrub.call_doublets(threshold=0.22)
scrub.plot_histogram();
scrublet = pd.DataFrame([scrub.doublet_scores_obs_, scrub.predicted_doublets_], columns=cistopic_obj.cell_names, index=['Doublet_scores_fragments', 'Predicted_doublets_fragments']).T
cistopic_obj.add_cell_data(scrublet, split_pattern = '-')
sum(cistopic_obj.cell_data.Predicted_doublets_fragments == True)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_path, "cistopic_obj.pkl"), "wb")
)

In [ ]:
!wget https://github.com/mimno/Mallet/releases/download/v202108/Mallet-202108-bin.tar.gz
!tar -xf Mallet-202108-bin.tar.gz

In [ ]:
# Load the pickled object
with open(os.path.join(out_path, "cistopic_obj.pkl"), "rb") as file:
    cistopic_obj = pickle.load(file)

In [ ]:
os.environ['MALLET_MEMORY'] = '900G'
from pycisTopic.lda_models import run_cgs_models_mallet
# Configure path Mallet
mallet_path="Mallet-202108/bin/mallet"
# Run models
models=run_cgs_models_mallet(
    cistopic_obj,
    n_topics=[15],
    n_cpu=20,
    n_iter=500,
    random_state=555,
    alpha=50,
    alpha_by_topic=True,
    eta=0.1,
    eta_by_topic=False,
    tmp_path=os.path.join(out_path, "mallet"),
    save_path=os.path.join(out_path, "mallet"),
    mallet_path=mallet_path,
)

In [ ]:
pickle.dump(
    models,
    open(os.path.join(out_path, "models.pkl"), "wb")
)

In [ ]:
import pickle
with open(os.path.join(out_path, "models.pkl"), "rb") as file:
    models = pickle.load(file)
with open(os.path.join(out_path, "cistopic_obj.pkl"), "rb") as file:
    cistopic_obj = pickle.load(file)

In [ ]:
from pycisTopic.lda_models import evaluate_models
model = evaluate_models(
    models,
    select_model = 15,
    return_model = True
)

In [ ]:
cistopic_obj.add_LDA_model(model)

In [ ]:
print(cistopic_obj.selected_model)


In [ ]:
cistopic_obj.cell_data['merged_id'] = cistopic_obj.cell_data['merged_id'].astype(str)
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_path, "cistopic_obj.pkl"), "wb")
)

In [ ]:
import pickle
with open(os.path.join(out_path, "cistopic_obj.pkl"), "rb") as file:
    cistopic_obj = pickle.load(file)

In [ ]:
from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)

In [ ]:
find_clusters(
    cistopic_obj,
    target  = 'cell',
    k = 10,
    res = [0.6, 1.2, 3],
    prefix = 'pycisTopic_',
    scale = True,
    split_pattern = '---'
)

In [ ]:
# Check if projections attribute exists and is a dictionary
if not hasattr(cistopic_obj, 'projections') or not isinstance(cistopic_obj.projections, dict):
    cistopic_obj.projections = {}

# Check if 'cell' key exists in projections dictionary and initialize it if necessary
if 'cell' not in cistopic_obj.projections:
    cistopic_obj.projections['cell'] = {}

In [ ]:
run_umap(
    cistopic_obj,
    target='cell',
    scale=True,
    n_neighbors=30,
    min_dist=0.5
)

plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['merged_id'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

In [ ]:
plot_topic(
    cistopic_obj,
    reduction_name = 'UMAP',
    target = 'cell',
    num_columns=5
)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_path, "cistopic_obj.pkl"), "wb")
)

In [ ]:
from pycisTopic.topic_binarization import binarize_topics
region_bin_topics_top_3k = binarize_topics(
    cistopic_obj, method='ntop', ntop = 3_000,
    plot=False, num_columns=5
)

In [ ]:
region_bin_topics_otsu = binarize_topics(
    cistopic_obj, method='otsu',
    plot=False, num_columns=5
)

In [ ]:
binarized_cell_topic = binarize_topics(
    cistopic_obj,
    target='cell',
    method='li',
    plot=False,
    num_columns=5, nbins=100)

In [ ]:
from pycisTopic.topic_qc import compute_topic_metrics, plot_topic_qc, topic_annotation
import matplotlib.pyplot as plt
from pycisTopic.utils import fig2img
topic_qc_metrics = compute_topic_metrics(cistopic_obj)

In [ ]:
fig_dict={}
fig_dict['CoherenceVSAssignments']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Log10_Assignments', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['AssignmentsVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Log10_Assignments', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSRegions_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Regions_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSMarginal_dist']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Marginal_topic_dist', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSGini_index']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Gini_index', var_color='Gini_index', plot=False, return_fig=True)

In [ ]:
# Plot topic stats in one figure
fig=plt.figure(figsize=(40, 43))
i = 1
for fig_ in fig_dict.keys():
    plt.subplot(2, 3, i)
    img = fig2img(fig_dict[fig_]) #To convert figures to png to plot together, see .utils.py. This converts the figure to png.
    plt.imshow(img)
    plt.axis('off')
    i += 1
plt.subplots_adjust(wspace=0, hspace=-0.70)
plt.show()

In [ ]:
topic_annot = topic_annotation(
    cistopic_obj,
    annot_var='merged_id',
    binarized_cell_topic=binarized_cell_topic,
    general_topic_thr = 0.2
)
topic_annot

In [ ]:
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)

In [ ]:
import numpy as np
imputed_acc_obj = impute_accessibility(
    cistopic_obj,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)

In [ ]:
pickle.dump(
    imputed_acc_obj,
    open(os.path.join(out_path, "imputed_acc_obj.pkl"), "wb")
)

In [ ]:
import pickle
with open(os.path.join(out_path, "imputed_acc_obj.pkl"), "rb") as file:
    imputed_acc_obj = pickle.load(file)

In [ ]:
# normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)

In [ ]:
# pickle.dump(
#     normalized_imputed_acc_obj,
#     open(os.path.join(out_path, "normalized_imputed_acc_obj.pkl"), "wb")
# )

In [ ]:
# import pickle
# with open(os.path.join(out_path, "normalized_imputed_acc_obj.pkl"), "rb") as file:
#     normalized_imputed_acc_obj = pickle.load(file)

In [ ]:
# variable_regions = find_highly_variable_features(
#     normalized_imputed_acc_obj,
#     min_disp = 0.05,
#     min_mean = 0.0125,
#     max_mean = 3,
#     max_disp = np.inf,
#     n_bins=20,
#     n_top_features=None,
#     plot=True
# )
# len(variable_regions)

In [ ]:
variable_regions = find_highly_variable_features(
    imputed_acc_obj,
    min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    n_top_features=None,
    plot=True
)
len(variable_regions)

In [ ]:
markers_dict= find_diff_features(
    cistopic_obj,
    imputed_acc_obj,
    variable='merged_id',
    var_features=variable_regions,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.5),
    n_cpu=16,
    _temp_dir="/cfs/klemming/projects/supr/secilmis",
    split_pattern = '-'
)

In [ ]:
pickle.dump(
    markers_dict,
    open(os.path.join(out_path, "markers_dict.pkl"), "wb")
)

In [ ]:
# from pycisTopic.clust_vis import plot_imputed_features
# plot_imputed_features(
#     cistopic_obj,
#     reduction_name='UMAP',
#     imputed_data=imputed_acc_obj,
#     features=[markers_dict[x].index.tolist()[0] for x in ["Oligodendrocytes_1dpi", "Astrocytes_1dpi", "Microglia_1dpi", "OPCs_1dpi", "Ependymal_1dpi", "Oligodendrocytes_28dpi", "OPCs_28dpi", "Microglia_28dpi", "Astrocytes_28dpi", "Ependymal_28dpi", "Oligodendrocytes_3dpi", "OPCs_3dpi", "Microglia_3dpi", "Astrocytes_3dpi", "Ependymal_3dpi", "Oligodendrocytes_7dpi", "Microglia_7dpi", "OPCs_7dpi", "Astrocytes_7dpi", "Ependymal_7dpi", "Oligodendrocytes_U", "OPCs_U", "Astrocytes_U", "Microglia_U", "Ependymal_U"]],
#     scale=False,
#     num_columns=3
# )

In [ ]:
print("Number of DARs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(markers_dict[x])}")

In [ ]:
os.makedirs(os.path.join(out_path, "region_sets"), exist_ok = True)
os.makedirs(os.path.join(out_path, "region_sets", "Topics_otsu"), exist_ok = True)
os.makedirs(os.path.join(out_path, "region_sets", "Topics_top_3k"), exist_ok = True)
os.makedirs(os.path.join(out_path, "region_sets", "DARs_cell_type"), exist_ok = True)

In [ ]:
from pycisTopic.utils import region_names_to_coordinates
for topic in region_bin_topics_otsu:
    region_names_to_coordinates(
        region_bin_topics_otsu[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_path, "region_sets", "Topics_otsu", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for topic in region_bin_topics_top_3k:
    region_names_to_coordinates(
        region_bin_topics_top_3k[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_path, "region_sets", "Topics_top_3k", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for cell_type in markers_dict:
    region_names_to_coordinates(
        markers_dict[cell_type].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_path, "region_sets", "DARs_cell_type", f"{cell_type}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
import pyranges as pr
from pycisTopic.gene_activity import get_gene_activity
chromsizes = pd.read_table(os.path.join(out_dir, "qc", "mm10.chrom_sizes_and_alias.tsv"))
chromsizes

In [ ]:
chromsizes.rename({"# ucsc": "Chromosome", "length": "End"}, axis = 1, inplace = True)
chromsizes["Start"] = 0
chromsizes = pr.PyRanges(chromsizes[["Chromosome", "Start", "End"]])
chromsizes

In [ ]:
pr_annotation = pd.read_table(
        os.path.join(out_dir, "qc", "tss.bed")
    ).rename(
        {"Name": "Gene", "# Chromosome": "Chromosome"}, axis = 1)
pr_annotation["Transcription_Start_Site"] = pr_annotation["Start"]
pr_annotation = pr.PyRanges(pr_annotation)
pr_annotation

In [ ]:
gene_act, weigths = get_gene_activity(
    imputed_acc_obj,
    pr_annotation,
    chromsizes,
    use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
    upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it
                             # these bp will be taken (1kbp here)
    downstream=[1000,100000], # Search space downstream
    distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
    decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
    extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for
                          #this weight)
    extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
    gene_size_weight=False, # Whether to add a weights based on the length of the gene
    gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                          #in the genome
    remove_promoters=False, # Whether to remove promoters when computing gene activity scores
    average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene
                          #activity score
    scale_factor=1, # Value to multiply for the final gene activity matrix
    extend_tss=[10,10], # Space to consider a promoter
    gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
    return_weights= True, # Whether to return the final weights
    project='Gene_activity') # Project name for the gene activity object

In [ ]:
DAG_markers_dict= find_diff_features(
    cistopic_obj,
    gene_act,
    variable='merged_id',
    var_features=None,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.5),
    n_cpu=20,
    _temp_dir="/cfs/klemming/projects/supr/secilmis",
    split_pattern = '-')

In [ ]:
pickle.dump(
    DAG_markers_dict,
    open(os.path.join(out_path, "DAG_markers_dict.pkl"), "wb")
)

In [ ]:
print("Number of DAGs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(DAG_markers_dict[x])}")

In [ ]:
from pycisTopic.loom import export_region_accessibility_to_loom, export_gene_activity_to_loom
cluster_markers = {'merged_id': markers_dict}
os.makedirs(os.path.join(out_path, "loom"), exist_ok=True)
export_region_accessibility_to_loom(
    accessibility_matrix = imputed_acc_obj,
    cistopic_obj = cistopic_obj,
    binarized_topic_region = region_bin_topics_otsu,
    binarized_cell_topic = binarized_cell_topic,
    selected_cells = cistopic_obj.projections['cell']['UMAP'].index.tolist(),
    out_fname = os.path.join(out_path, "loom", "multiome_pycisTopic_region_accessibility.loom"),
    cluster_annotation = ['merged_id'],
    cluster_markers = cluster_markers,
    tree_structure = ('multiome', 'pycisTopic', 'noDBL_all'),
    title = 'multiome - Region accessibility all',
    nomenclature = "mm10",
    split_pattern = '-'
)

In [ ]:
export_gene_activity_to_loom(
    gene_activity_matrix = gene_act,
    cistopic_obj = cistopic_obj,
    out_fname = os.path.join(out_path, "loom", "multiome_pycisTopic_gene_activity.loom"),
    cluster_annotation = ['merged_id'],
    cluster_markers = cluster_markers,
    tree_structure = ('multiome', 'pycisTopic', 'ATAC'),
    title = 'multiome - Gene activity',
    nomenclature = "mm10",
    split_pattern = '-'
)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_path, "cistopic_obj.pkl"), "wb")
)